In [ ]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. 
# This directory will be recovered automatically after resetting environment. 
!ls /home/aistudio/data

data101280


In [ ]:
# 查看工作区文件, 该目录下的变更将会持久保存. 请及时清理不必要的文件, 避免加载过慢.
# View personal work directory. 
# All changes under this directory will be kept even after reset. 
# Please clean unnecessary files in time to speed up environment loading. 
!ls /home/aistudio/work

In [ ]:
# 如果需要进行持久化安装, 需要使用持久化路径, 如下方代码示例:
# If a persistence installation is required, 
# you need to use the persistence path as the following: 
# !mkdir /home/aistudio/external-libraries
# !pip install beautifulsoup4 -t /home/aistudio/external-libraries

In [ ]:
# 同时添加如下代码, 这样每次环境(kernel)启动的时候只要运行下方代码即可: 
# Also add the following code, 
# so that every time the environment (kernel) starts, 
# just run the following code: 
import sys 
sys.path.append('/home/aistudio/external-libraries')

In [ ]:
#初始化环境,导包

# 忽视警告
import warnings
warnings.filterwarnings('ignore')
# 多行展示
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
# pandas/numpy 工具包
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# K折交叉验证
import sklearn
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import accuracy_score
# XGboost模型
import xgboost as xgb
from xgboost import plot_importance
# LGBM模型
import lightgbm as lgb

In [ ]:
# 读取数据集
train = pd.read_csv('./data/data101280/train.csv')
test = pd.read_csv('./data/data101280/test1.csv')

In [ ]:
# 除去原有序列号, 划分X, y, X_test
X = train.iloc[:, 1:].drop(['label'], axis=1)
y = train['label']
X_test  = test.iloc[:, 1:]

In [ ]:
# 查看数据结构
X.head()
print('y.head()',"------"*20)
y.head()
print('X_test.head()',"------"*20)
X_test.head()
print("------"*20)
X.info()
print("------"*20)
X.nunique()
print("------"*20)

,android_id,apptype,carrier,dev_height,dev_ppi,dev_width,lan,media_id,ntt,os,osv,package,sid,timestamp,version,fea_hash,location,fea1_hash,cus_type
0,316361,1199,46000.0,0.0,0.0,0.0,NaN,104,6.0,android,9,18,1438873,1.559893e+12,8,2135019403,0,2329670524,601
1,135939,893,0.0,0.0,0.0,0.0,NaN,19,6.0,android,8.1,0,1185582,1.559994e+12,4,2782306428,1,2864801071,1000
2,399254,821,0.0,760.0,0.0,360.0,NaN,559,0.0,android,8.1.0,0,1555716,1.559837e+12,0,1392806005,2,628911675,696
3,68983,1004,46000.0,2214.0,0.0,1080.0,NaN,129,2.0,android,8.1.0,0,1093419,1.560042e+12,0,3562553457,3,1283809327,753
4,288999,1076,46000.0,2280.0,0.0,1080.0,zh-CN,64,2.0,android,8.0.0,0,1400089,1.559867e+12,5,2364522023,4,1510695983,582


y.head() ------------------------------------------------------------------------------------------------------------------------


0    1
1    1
2    1
3    0
4    1
Name: label, dtype: int64

X_test.head() ------------------------------------------------------------------------------------------------------------------------


,android_id,apptype,carrier,dev_height,dev_ppi,dev_width,lan,media_id,ntt,os,osv,package,sid,timestamp,version,fea_hash,location,fea1_hash,cus_type
0,317625,1181,46000.0,2196.0,2.0,1080.0,CN,639,2.0,Android,8.1.0,188,1440682,1.559872e+12,7,1672223856,57,3872258917,658
1,435108,944,46003.0,2280.0,3.0,1080.0,zh-CN,704,6.0,Android,8.1.0,221,1606824,1.559739e+12,3,3767901757,23,129322164,943
2,0,1106,46000.0,0.0,0.0,0.0,NaN,39,2.0,android,5.1,1562,1774642,1.559614e+12,0,454638703,30,4226678391,411
3,451504,761,46000.0,1344.0,0.0,720.0,NaN,54,2.0,android,7.1.1,9,1742535,1.559668e+12,0,1507622951,65,3355419572,848
4,0,1001,46000.0,665.0,0.0,320.0,zh-CN,29,5.0,Android,8.1.0,4,1689686,1.559694e+12,0,4116351093,148,2644467751,411


------------------------------------------------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 19 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   android_id  500000 non-null  int64  
 1   apptype     500000 non-null  int64  
 2   carrier     500000 non-null  float64
 3   dev_height  500000 non-null  float64
 4   dev_ppi     500000 non-null  float64
 5   dev_width   500000 non-null  float64
 6   lan         316720 non-null  object 
 7   media_id    500000 non-null  int64  
 8   ntt         500000 non-null  float64
 9   os          500000 non-null  object 
 10  osv         493439 non-null  object 
 11  package     500000 non-null  int64  
 12  sid         500000 non-null  int64  
 13  timestamp   500000 non-null  float64
 14  version     500000 non-null  object 
 15  fea_hash    500000 non-null  object 
 16  locatio

android_id    362258
apptype           89
carrier            5
dev_height       798
dev_ppi           92
dev_width        346
lan               21
media_id         284
ntt                8
os                 2
osv              154
package         1950
sid           500000
timestamp     500000
version           22
fea_hash      402980
location         332
fea1_hash       4959
cus_type          58
dtype: int64

------------------------------------------------------------------------------------------------------------------------


In [ ]:
# 去除sid
X = X.drop(['sid'], axis=1)
X_test = X_test.drop(['sid'], axis=1)

**查看缺失字段**

In [ ]:
temp = X.isnull().sum()
temp[temp > 0].index

Index(['lan', 'osv'], dtype='object')

In [ ]:
#处理lan字段
X['lan'].fillna('NaN', inplace=True)
X_test['lan'].fillna('NaN', inplace=True)
X_test['lan']

0            CN
1         zh-CN
2           NaN
3           NaN
4         zh-CN
          ...  
149995    zh-CN
149996    zh-CN
149997    zh-CN
149998    zh-CN
149999    zh-CN
Name: lan, Length: 150000, dtype: object

In [ ]:
# 处理osv
# temp = X['osv'].groupby(X['osv']).count().reset_index(name='count')
# print(temp[temp['count']>=50])
# print(X[X['osv'].value_counts()>=50])
# X.isna().sum()

X['osv'].fillna('8.1.0', inplace=True)
X_test['osv'].fillna('8.1.0', inplace=True)


In [ ]:
# X.info()
X.select_dtypes(include='object').columns

Index(['lan', 'os', 'osv', 'version', 'fea_hash'], dtype='object')

In [ ]:
# 手动编码, 进行字典映射, 将相近类别放到一起
lan_map = {'zh':1, 'ZH':2, 'zh-cn':3, 'zh-CN':4, 'zh_CN':5, 'Zh-CN':6, 'zh_CN_#Hans':7, 'cn':8, 'CN':9, 'zh-HK':10,
 'zh-TW':11, 'tw':12, 'TW':13, 'zh-MO':14, 'en':15, 'en-US':16, 'en-GB':17, 'ko':18, 'it':19, 'ja':20, 'mi':21, 'NaN':22}
X['lan'] = X['lan'].map(lan_map)
X_test['lan'] = X_test['lan'].map(lan_map)

In [ ]:
# 映射处理
X['os'] = X['os'].map(lambda x: 1 if x == 'Android' else 0)
X_test['os'] = X_test['os'].map(lambda x: 1 if x == 'Android' else 0)

In [ ]:
X['osv'].value_counts()
X['osv'].value_counts().index.tolist()

8.1.0                    157980
9                         71332
6.0.1                     31714
7.1.1                     26503
9.0.0                     24385
                          ...  
4                             1
Android 4.2.9                 1
2.0.1                         1
5.1.1-F-20180719.9007         1
2.2.2                         1
Name: osv, Length: 154, dtype: int64

['8.1.0',
 '9',
 '6.0.1',
 '7.1.1',
 '9.0.0',
 '6.0',
 '5.1',
 '8.1',
 '5.1.1',
 '8.0.0',
 '7.1.2',
 '4.4.4',
 '7.0',
 'Android_8.1.0',
 'Android_9',
 '5.0.2',
 '7.9.0',
 '7.9.2',
 '8.0',
 '7.1',
 '9.0',
 '3.8.6',
 '4.4.2',
 '6.0.0',
 '5.1.0',
 '4.1',
 '2.3',
 '5.0',
 '4.3',
 '7.0.0',
 'Android_8.0.0',
 '4.4',
 'Android_7.1.1',
 '3.9.0',
 'Android_6.0.1',
 '7.8.9',
 '4.2.2',
 '7.8.7',
 '8.0.1',
 '5.0.1',
 'Android_5.1.1',
 'Android_7.1.2',
 '6.1',
 '7.8.0',
 '3.8.4',
 'Android_5.1',
 '8',
 '4.1.2',
 'Android_6.0',
 '5.0.0',
 '7.7.7',
 'Android_7.0',
 '7.8.2',
 '3.8.0',
 '21100',
 '7.7.3',
 'Android_4.4.4',
 '7.2.1',
 '4.3.0',
 '7',
 '4.2.1',
 '7.7.5',
 '6.1.0',
 '5.2',
 '7.8.4',
 '7.8.5',
 '7.7.2',
 '21000',
 '5',
 '4.1.1',
 '7.6.7',
 '7.7.0',
 '3.8.3',
 '6.0_23',
 '7.6.8',
 '3.7.6',
 '7.6.4',
 '7.8.8',
 'Android_5.0.2',
 '9.0.5',
 '3.7.8',
 '7.1.3',
 '7.7.4',
 '7.1.0',
 '7.6.9',
 '4.0.3',
 '4.0.4',
 '4.2',
 'Android_5.0',
 '4.4.5',
 '7.8.6',
 '6.2.1',
 'Android_4.4.2',
 '3.1.1',
 '4.4

In [ ]:
# 特征转换
def osv_trans(x):
    x = str(x)
    x = x.replace('Android_', '').replace('Android ', '').replace('W', '')
    if x.find('.') > 0: 
        temp_index1 = x.find('.')
        temp_index2 = len(x)
        if x.find(' ') > 0: temp_index2 = x.find(' ')
        if x.find('-') > 0: temp_index2 = x.find('-') if x.find('-') < temp_index2 else temp_index2
        x = x[:temp_index1]+'.'+x[temp_index1:temp_index2].replace('.','')
    try: return float(x)
    except: return 0


X['osv'] = X['osv'].apply(osv_trans)
X_test['osv'] = X_test['osv'].apply(osv_trans)

In [ ]:
# 欺诈正常比
temp = pd.DataFrame(columns=[0,1])
# 所有正常的数据
temp0 = X[y==0]
temp[0] = temp0['osv'].value_counts() / len(temp0) # 在正常的里面, 每个版本所占的百分比
# 所有异常的数据
temp1 = X[y==1]
temp[1] = temp1['osv'].value_counts() / len(temp1)


# 构造特征比, 选择出主要特征,其余非主要特征归零。
temp[2] = temp[1] / temp[0]
key_features = temp[temp[2]>10].sort_values(2, ascending=False).index
key_features
X['osv_is_key_feature'] = X['osv'].apply(lambda x: 1 if x in key_features else 0)
X_test['osv_is_key_feature'] = X_test['osv'].apply(lambda x: 1 if x in key_features else 0)

Float64Index([   7.77,    7.21,    7.75,    7.85,    7.87,     3.8,    7.67,
                  3.9,     2.3,    8.01,     7.9,    7.64,    3.84,    7.89,
              21100.0,    7.92,     4.1,    7.72,    7.82,     7.8,    3.86,
                  7.7,    7.84,    7.68, 21000.0,    7.86,    7.73,     6.1,
                 3.83,    3.78,    7.74],
             dtype='float64')

In [ ]:
def version_trans(x):
    if x == 'V3' or x == 'GA3': return 3
    elif x == 'v1': return 1
    elif x == 'P_Final_6' or x == 'V6': return 6
    elif x == 'GA2' or x == 'V2': return 2
    else: return int(x)
X['version'] = X['version'].apply(version_trans)
X_test['version'] = X_test['version'].apply(version_trans)

In [ ]:
X['osv_version'] = X['osv'] - X['version']
X_test['osv_version'] = X_test['osv'] - X_test['version']

In [ ]:
X['fea_hash'] = X['fea_hash'].map(lambda x: len(str(x)))
X_test['fea_hash'] = X_test['fea_hash'].map(lambda x: len(str(x)))
X['fea_hash_len'] = X['fea_hash'].apply(lambda x: x if x <= 10 else 0)
X_test['fea_hash_len'] = X_test['fea_hash'].apply(lambda x: x if x <= 10 else 0)

In [ ]:
X['fea1_hash'] = X['fea1_hash'].map(lambda x: len(str(x)))
X_test['fea1_hash'] = X_test['fea1_hash'].map(lambda x: len(str(x)))
X['fea1_hash_len'] = X['fea1_hash'].apply(lambda x: x if x <= 10 else 0)
X_test['fea1_hash_len'] = X_test['fea1_hash'].apply(lambda x: x if x <= 10 else 0)

In [ ]:
# 构造面积特征
X['area'] = X['dev_height'] * X['dev_width']
X_test['area'] = X_test['dev_height'] * X_test['dev_width']
# 构造高宽比
X['height_width'] = X['dev_height'] / X['dev_width']
X['height_width'].fillna(0, inplace=True)
X_test['height_width'] = X_test['dev_height'] / X_test['dev_width']
X_test['height_width'].fillna(0, inplace=True)
# 像素点数
X['pix'] = X['dev_ppi'] * X['area']
X_test['pix'] = X_test['dev_ppi'] * X_test['area']

In [ ]:
from datetime import datetime
X['timestamp'] = X['timestamp'].apply(lambda x: datetime.fromtimestamp(x / 1000))
X_test['timestamp'] = X_test['timestamp'].apply(lambda x: datetime.fromtimestamp(x / 1000))

In [ ]:
temp = pd.DatetimeIndex(X['timestamp'])
X['year'] = temp.year
X['month'] = temp.month
X['day'] = temp.day
X['week_day'] = temp.weekday
X['hour'] = temp.hour
X['minute'] = temp.minute

start_time = X['timestamp'].min()
X['time_diff'] = X['timestamp'] - start_time
X['time_diff'] = X['time_diff'].dt.days + X['time_diff'].dt.seconds/3600/24

temp = pd.DatetimeIndex(X_test['timestamp'])
X_test['year'] = temp.year
X_test['month'] = temp.month
X_test['day'] = temp.day
X_test['week_day'] = temp.weekday
X_test['hour'] = temp.hour
X_test['minute'] = temp.minute

X_test['time_diff'] = X_test['timestamp'] - start_time
X_test['time_diff'] = X_test['time_diff'].dt.days + X_test['time_diff'].dt.seconds/3600/24

X = X.drop(['timestamp'], axis=1)
X_test = X_test.drop(['timestamp'], axis=1)

In [ ]:
# 使用lgb构造baselight
# model = lgb.LGBMClassifier(num_leaves=2**5-1,
#                            reg_alpha=0.25,
#                            reg_lambda=0.25,
#                            objective='binary',
#                            max_depth=-1, 
#                            learning_rate=0.005,
#                            min_child_samples=3, 
#                            random_state=2021,
#                            n_estimators=10000,
#                            subsample=1, 
#                            colsample_bytree=1)
# %time model.fit(X,y)
# res = pd.DataFrame(test['sid'])
# res['label'] = model.predict(X_test)
# res

In [34]:
# max_depth=10, learning_rate=0.005, n_estimators=5800,objective='binary:logistic', tree_method='exact',subsample=0.8, colsample_bytree=0.8,
# min_child_weight=3, eval_metric='auc', reg_lambda=0.5 ------> 0.89176
# model_xgb = xgb.XGBClassifier(
#                 max_depth=10, learning_rate=0.005, n_estimators=5300,
#                 objective='binary:logistic', tree_method='gpu_hist',
#                 subsample=0.8, colsample_bytree=0.8,
#                 min_child_weight=3, eval_metric='auc', reg_lambda=0.5)
# %time model_xgb.fit(X,y)


# max_depth=10, learning_rate=0.005, n_estimators=5800,objective='binary:logistic', tree_method='exact',subsample=0.8, colsample_bytree=0.8,
# min_child_weight=3, eval_metric='auc', reg_lambda=0.5 ------> 0.891773

# tree_method 选择exact 会比 gpu_hist 时间长换正确率
model_xgb1 = xgb.XGBClassifier(
                max_depth=10, learning_rate=0.005, n_estimators=5800,
                objective='binary:logistic', tree_method='exact',
                subsample=0.8, colsample_bytree=0.8,
                min_child_weight=3, eval_metric='auc', reg_lambda=0.5)
# %time model_xgb1.fit(X,y)

# 多折交叉验证 ensemble model
def xgb_model(clf, X, y, X_test, n_splits):
    # 分层K折验证
    skf = StratifiedKFold(n_splits=n_splits, random_state=2021, shuffle=True)
    mean_acc = 0
    acc_sum = 0
    prob = []
    for k, (train_index, val_index) in enumerate(skf.split(X, y)):
        # 获取训练集和验证集
        train_x = X.iloc[train_index]
        train_y = y.iloc[train_index]
        val_x = X.iloc[val_index]
        val_y = y.iloc[val_index]
        # 开始训练
        clf = clf.fit(train_x, train_y)
        # 开始验证
        val_y_pred = clf.predict(val_x)
        acc_val = accuracy_score(val_y, val_y_pred)
        acc_sum += acc_val
        mean_acc_temp = acc_sum / (k+1)
        print(f'第 {k+1} 个子模型, accuracy = {acc_val}, mean_acc = {mean_acc_temp}')
        mean_acc += acc_val / n_splits
        # 开始测试
        y_test_pred = clf.predict_proba(X_test)
        prob.append(y_test_pred)
    mean_prob = sum(prob) / n_splits
    return mean_acc, mean_prob

# 十折交叉验证
acc, result = xgb_model(model_xgb1, X, y, X_test, 10)
acc
result
res = pd.DataFrame(test['sid'])
res['label'] = np.where(result[:,1] > 0.5, 1, 0)
res
res.to_csv('./result/xgb1_10_5300.csv', index=False)

In [ ]:
#尝试提取 0,6测试结果发现过拟合,舍弃
def xgb_model_0_6(clf, X, y, X_test, n_splits):
    # 分层K折验证
    skf = StratifiedKFold(n_splits=n_splits, random_state=2021, shuffle=True)
    mean_acc = 0
    acc_sum = 0
    prob = []
    # for k, (train_index, val_index) in enumerate(skf.split(X, y)):
    #     train_sets = []
    #     if k == 0 or k == 6:
    #         train_sets.append((train_index, val_index))
    for k, (train_index, val_index) in enumerate([list(skf.split(X,y))[0],list(skf.split(X,y))[6]]):
        # 获取训练集和验证集
        train_x = X.iloc[train_index]
        train_y = y.iloc[train_index]
        val_x = X.iloc[val_index]
        val_y = y.iloc[val_index]
        # 开始训练
        clf = clf.fit(train_x, train_y)
        # 开始验证
        val_y_pred = clf.predict(val_x)
        acc_val = accuracy_score(val_y, val_y_pred)
        acc_sum += acc_val
        mean_acc_temp = acc_sum / (k+1)
        print(f'第 {k+1} 个子模型, accuracy = {acc_val}, mean_acc = {mean_acc_temp}',end='\n')
        mean_acc += acc_val / 2
        # 开始测试
        y_test_pred = clf.predict_proba(X_test)
        prob.append(y_test_pred)
    mean_prob = sum(prob) / 2
    return mean_acc, mean_prob

# acc, result = xgb_model_0_6(model_xgb, X, y, X_test, 10)
# acc
# result
# res = pd.DataFrame(test['sid'])
# res['label'] = np.where(result[:,1] > 0.5, 1, 0)
# res
# res.to_csv('./result/xgb_0_6.csv', index=False)

CPU times: user 2min 4s, sys: 18.6 s, total: 2min 22s
Wall time: 2min 22s
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, eval_metric='auc',
              gamma=0, gpu_id=0, importance_type='gain',
              interaction_constraints='', learning_rate=0.005, max_delta_step=0,
              max_depth=10, min_child_weight=3, missing=nan,
              monotone_constraints='()', n_estimators=5300, n_jobs=24,
              num_parallel_tree=1, random_state=0, reg_alpha=0, reg_lambda=0.5,
              scale_pos_weight=1, subsample=0.8, tree_method='gpu_hist',
              validate_parameters=1, verbosity=None)
第 1 个子模型, accuracy = 0.89284, mean_acc = 0.89284
第 2 个子模型, accuracy = 0.8913, mean_acc = 0.8920699999999999
第 3 个子模型, accuracy = 0.8911, mean_acc = 0.8917466666666666
第 4 个子模型, accuracy = 0.89176, mean_acc = 0.8917499999999999
第 5 个子模型, accuracy = 0.89096, mean_acc = 0.8915919999999999
第 6 个子模型, accuracy = 0.89282, mean_acc = 0.8917966666666667
第 7 个子模型, accuracy = 0.8932, mean_acc = 0.8919971428571429
第 8 个子模型, accuracy = 0.88998, mean_acc = 0.891745
第 9 个子模型, accuracy = 0.89084, mean_acc = 0.8916444444444446
第 10 个子模型, accuracy = 0.89268, mean_acc = 0.8917480000000001

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 